In [0]:
import math,random
from pyspark.sql.functions import *
raw_data = sc.textFile("dbfs:/FileStore/shared_uploads/sxr230004@utdallas.edu/soc_LiveJournal1Adj-2.txt")
data = raw_data.map(lambda x: x.split("\t"))

In [0]:
data.take(10)

Out[13]: [['0',
  '1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94'],
 ['1',
  '0,5,20,135,2409,8715,8932,10623,12347,12846,13840,13845,14005,20075,21556,22939,23520,28193,29724,29791,29826,30691,31232,31435,32317,32489,34394,35589,35605,35606,35613,35633,35648,35678,38737,43447,44846,44887,49226,49985,623,629,4999,6156,13912,14248,15190,17636,19217,20074,27536,29481,29726,29767,30257,33060,34250,34280,34392,34406,34418,34420,34439,34450,34651,45054,49592'],
 ['2',
  '0,117,135,1220,2755,12453,24539,24714,41456,45046,49927,6893,13795,16659,32828,41878'],
 ['3', '0,12,41,55,1532,12636,13185,27552,38737'],
 ['4',
  '0,8,14,15,18,27,72,80,15326,19068,19079,24596,42697,46126,74,77,33269,38792,38822'],
 ['5',
  '0,1,20,2022,22939,23527,30257,32503,35633,41457,43262,4484

In [0]:
#collecting userids
userids = data.keys().collect()
#Flat mapping each userid(a[0]) to its friend list(a[1]) and reverse mapping to create key value pair with friend as key and userid as value
friendskey = data.map(lambda a : (a[0],list(a[1].split(',')))).flatMap(lambda a:[(a[0],i)for i in a[1]]).map( lambda a: (a[1],a[0])) 
friendskey.take(3)
friendskey.persist()

Out[14]: PythonRDD[70] at RDD at PythonRDD.scala:58

In [0]:
#joining the friends key rdd(friend,user) with original data(user,list of friends) to known friend list between 2 user ids
rd1.persist()
allfriends = rd1.map(lambda x:(x[1][0],(x[0],x[1][1]))).map(lambda x:(x[0],x[1][1])).reduceByKey(lambda x,y: (x +","+ y))
allfriends.take(3)
allfriends.persist()
 

Out[15]: PythonRDD[76] at RDD at PythonRDD.scala:58

In [0]:
#creating rdd that contains each (userid ,  its friends and friends of friends)
rd1  =  friendskey.join(data)

In [0]:
rd1.take(5)
rd1.persist()
#joining the friends key rdd(friend,user) with original data(user,list of friends) to known friend list between 2 user ids
allfriends = rd1.map(lambda x:(x[1][0],(x[0],x[1][1]))).map(lambda x:(x[0],x[1][1])).reduceByKey(lambda x,y: (x +","+ y))
allfriends.take(3)
allfriends.persist()


Out[17]: PythonRDD[91] at RDD at PythonRDD.scala:58

In [0]:
#counting how many mutal friends between each pair userids
count_mutuals = allfriends.flatMap(lambda x:((x[0],i) for i in x[1].split(','))).subtract(friendskey).map(lambda x:(x[0],(x[1],1))).filter(lambda x: x[1][0]!=x[0]).map(lambda x:((x[0]+"_"+x[1][0]),x[1][1])).reduceByKey(lambda x,y:x+y)

In [0]:
count_mutuals.sortByKey().sortBy(lambda v: -v[1]).collect()
count_mutuals.persist()

Out[19]: PythonRDD[116] at RDD at PythonRDD.scala:58

In [0]:
def ten_recommandations(v):
    return v.split(",")[:10]
recommendation_system = count_mutuals.map(lambda x: (x[0][0:x[0].index("_")],(x[0][x[0].index("_")+1:len(x[0])], x[1]))).sortBy(lambda x: -x[1][1]).map(lambda x:(x[0],(x[1][0]))).reduceByKey(lambda x,y:((x+","+y))).map(lambda x:(x[0],ten_recommandations(x[1])))
random_users = random.sample(userids,10)
print(random_users)
rand_users_friends = recommendation_system.filter(lambda x: x[0] in random_users).sortByKey().collect()

['3959', '41411', '14409', '22818', '27067', '43358', '28373', '33684', '15246', '24341']


In [0]:
for x in rand_users_friends:
    # Print the UserID followed by a tab character
    print(f"{x[0]}\t", end="")
    if x[1]:
        # Print the list of friends' User IDs, comma-separated
        friend_ids = ",".join(map(str, x[1][0:-1]))
        print(f"{friend_ids},{x[1][-1]}")

15246	26114,15256,43014,31280,17478,31304,31236,1966,31225,31254
24341	24350,24626,40439,48364,24335,24336,24355,14730,37343,24347
27067	27119,27101,49948,27077,27060,10380,27087,27096,32178,9569
28373	32096,19350,6452,9154,11132,33274,40490,39751,46428,31437
33684	30486,30466,30448,21240,30386,21693,30370,33702,30470,30339
3959	3890,3977,3907,4131,3884,3975,3916,3924,4015,4134
41411	41406,41405,41458,41413,6451,41433,41414,38128,41409,22927
43358	43181,43136,43338,17070,43190,43650,8501,243,43171,5874
